In [1]:
import pandas as pd

!unzip ../Data/1ADQ_AV2PooledComplete.zip
df = pd.read_csv('1ADQ_AV2PooledComplete.txt', sep='\t')
!rm 1ADQ_AV2PooledComplete.txt

print(df.shape[0])
df.head()


Archive:  ../Data/1ADQ_AV2PooledComplete.zip
  inflating: 1ADQ_AV2PooledComplete.txt  
47142745


,ID_slide_Variant,CDR3,Best,Slide,Energy,Structure,Antigen
0,5455_03a_00a,CARYITLRWYFDVW,True,CARYITLRWYF,-89.79,132966-BDUDDRDURR,1ADQ_A+V25A
1,5455_03a_01a,CARYITLRWYFDVW,False,ARYITLRWYFD,-83.28,124837-RRURSUUSLD,1ADQ_A+V25A
2,5455_03a_02a,CARYITLRWYFDVW,False,RYITLRWYFDV,-79.07,128869-BRSRSRRDLU,1ADQ_A+V25A
3,5455_03a_03a,CARYITLRWYFDVW,False,YITLRWYFDVW,-85.20,132966-BDRSUUSLUL,1ADQ_A+V25A
4,19272_00a_00a,CARRGLLLDFDVW,False,CARRGLLLDFD,-84.33,132966-BDSUURSUUR,1ADQ_A+V25A


In [2]:
df1 = df[['Antigen', 'Slide', 'Energy']]
df2 = df1.groupby(by='Antigen', as_index=False).count()

Ag_lst = list(df2.Antigen)
mut_lst = '+'.join(['+'.join(x.split('+')[1:]) for x in Ag_lst]).split('+')
mut_lst = list(set([x[:-1] for x in mut_lst]))

df3 = pd.DataFrame(mut_lst)
df3['AA'] = df3.apply(lambda x: x[0][0], axis=1)
df3['position'] = df3.apply(lambda x: int(x[0][1:]), axis=1)
df3 = df3.sort_values(by='position').reset_index(drop=True)
BaseAgSeq = ''.join(df3.AA)
pos_dct = {df3.loc[i].position: i for i in range(df3.shape[0])}
print(BaseAgSeq)


VDVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL


In [3]:
def newseq(seq, mutations, pos_dct):
    mutations = mutations.split('+')[1:]
    for x in mutations:
        mut = x[-1]
        pos = int(x[1:-1])
        seq = seq[:pos_dct[pos]]+mut+seq[pos_dct[pos]+1:]
    return seq

Ag_dct = {}
for mutations in Ag_lst:
    Ag_dct[mutations] = newseq(seq = BaseAgSeq, mutations = mutations, pos_dct = pos_dct)

df1['AgSeq'] = df1.apply(lambda x: Ag_dct[x.Antigen], axis=1)
df1.to_csv('../Data/1ADQ.tsv', sep='\t', index=None)
df1.head()


/var/folders/8m/wb_394bd6wg7gpzy45qbqwrm0000gn/T/ipykernel_10303/3106258804.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['AgSeq'] = df1.apply(lambda x: Ag_dct[x.Antigen], axis=1)


,Antigen,Slide,Energy,AgSeq
0,1ADQ_A+V25A,CARYITLRWYF,-89.79,ADVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL
1,1ADQ_A+V25A,ARYITLRWYFD,-83.28,ADVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL
2,1ADQ_A+V25A,RYITLRWYFDV,-79.07,ADVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL
3,1ADQ_A+V25A,YITLRWYFDVW,-85.20,ADVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL
4,1ADQ_A+V25A,CARRGLLLDFD,-84.33,ADVQQVWVDVQTKPERVNYVLPPEQVLKGLLYRL
